In [ ]:
import arcpy
import os
import netCDF4 as nc
from datetime import datetime
from collections import defaultdict

arcpy.env.addOutputsToMap = False  # Prevents adding layers to the map in ArcGIS

# User-defined inputs
netCDF_file = input("Enter the full path to your NetCDF file (.nc): ")
output_folder = input("Enter the output folder path for raster files: ")

# NetCDF parameters
variable = "depth"
x_dimension = "x"
y_dimension = "y"
time_dimension = "time"

# Ensure output folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Open the NetCDF file and read time values
dataset = nc.Dataset(netCDF_file)
time_var = dataset.variables[time_dimension]
time_units = time_var.units

# Convert NetCDF time values to datetime objects
time_values = nc.num2date(time_var[:], units=time_units)

# Dictionary to store rasters by month
monthly_rasters = defaultdict(list)

# Iterate over time values to create raster layers
for i, time in enumerate(time_values):
    date_str = time.strftime("%m/%d/%Y")  # Format time as 'MM/DD/YYYY'

    out_raster_layer = f"depth_{i+1}"
    dimension_values = f"time '{time.strftime('%m/%d/%Y %I:%M:%S %p')}'"  # Preserve NetCDF query format

    # Create NetCDF raster layer
    arcpy.md.MakeNetCDFRasterLayer(
        in_netCDF_file=netCDF_file,
        variable=variable,
        x_dimension=x_dimension,
        y_dimension=y_dimension,
        out_raster_layer=out_raster_layer,
        band_dimension="",
        dimension_values=dimension_values,
        value_selection_method="BY_VALUE",
        cell_registration="CENTER",
    )

    # Save raster to the output folder
    output_raster_path = os.path.join(output_folder, f"{out_raster_layer}.tif")
    arcpy.management.CopyRaster(out_raster_layer, output_raster_path)

    # Organize raster paths by month
    raster_date = datetime.strptime(date_str, "%m/%d/%Y")
    month_key = raster_date.strftime("%Y-%m")
    monthly_rasters[month_key].append(output_raster_path)

    print(f"Created and saved: {output_raster_path}")

print("All raster layers created successfully.")